# Preprocessing

#### This notebook contains all the necessary work to prepare the dataset for later analysis.

In [1]:
import pandas as pd
import numpy as np
from processing_tools import * #Located within the same directory
from sklearn.model_selection import train_test_split

In [2]:
path = '../Datasets/speed_dating.csv'
df = pd.read_csv(path,low_memory=False)

In [3]:
df.replace('?',np.nan,inplace=True) #Nulls are coded as "?", so I just turn them into np.nan

In [4]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8378 entries, 0 to 8377
Data columns (total 123 columns):
 #    Column                           Dtype 
---   ------                           ----- 
 0    has_null                         int64 
 1    wave                             int64 
 2    gender                           object
 3    age                              object
 4    age_o                            object
 5    d_age                            int64 
 6    d_d_age                          object
 7    race                             object
 8    race_o                           object
 9    samerace                         int64 
 10   importance_same_race             object
 11   importance_same_religion         object
 12   d_importance_same_race           object
 13   d_importance_same_religion       object
 14   field                            object
 15   pref_o_attractive                object
 16   pref_o_sincere                   object
 17   pref_o_intel

Most columns are of type "object", but in reality, they should be numeric. This is probably caused by the 
use of "?" as the null representation.To fix it, I created a function called "convert_str" and applied 
it to the whole dataframe.

In [5]:
df = df.applymap(convert_str)

Some column names have typos, so I went to the dataset description site and grabbed all the columns from there to check them against mine. The dataset description can be found here:
[Speed Dating Dataset](https://www.openml.org/search?type=data&sort=runs&status=active&id=40536)

In [6]:
cols = '''* gender: Gender of self  
 * age: Age of self  
 * age_o: Age of partner  
 * d_age: Difference in age  
 * race: Race of self  
 * race_o: Race of partner  
 * samerace: Whether the two persons have the same race or not.  
 * importance_same_race: How important is it that partner is of same race?  
 * importance_same_religion: How important is it that partner has same religion?  
 * field: Field of study  
 * pref_o_attractive: How important does partner rate attractiveness  
 * pref_o_sinsere: How important does partner rate sincerity  
 * pref_o_intelligence: How important does partner rate intelligence  
 * pref_o_funny: How important does partner rate being funny  
 * pref_o_ambitious: How important does partner rate ambition  
 * pref_o_shared_interests: How important does partner rate having shared interests  
 * attractive_o: Rating by partner (about me) at night of event on attractiveness  
 * sincere_o: Rating by partner (about me) at night of event on sincerity  
 * intelligence_o: Rating by partner (about me) at night of event on intelligence  
 * funny_o: Rating by partner (about me) at night of event on being funny  
 * ambitous_o: Rating by partner (about me) at night of event on being ambitious  
 * shared_interests_o: Rating by partner (about me) at night of event on shared interest  
 * attractive_important: What do you look for in a partner - attractiveness  
 * sincere_important: What do you look for in a partner - sincerity  
 * intellicence_important: What do you look for in a partner - intelligence  
 * funny_important: What do you look for in a partner - being funny  
 * ambtition_important: What do you look for in a partner - ambition  
 * shared_interests_important: What do you look for in a partner - shared interests  
 * attractive: Rate yourself - attractiveness  
 * sincere: Rate yourself - sincerity   
 * intelligence: Rate yourself - intelligence   
 * funny: Rate yourself - being funny   
 * ambition: Rate yourself - ambition  
 * attractive_partner: Rate your partner - attractiveness  
 * sincere_partner: Rate your partner - sincerity   
 * intelligence_partner: Rate your partner - intelligence   
 * funny_partner: Rate your partner - being funny   
 * ambition_partner: Rate your partner - ambition   
 * shared_interests_partner: Rate your partner - shared interests  
 * sports: Your own interests [1-10]  
 * tvsports  
 * exercise  
 * dining  
 * museums  
 * art  
 * hiking  
 * gaming  
 * clubbing  
 * reading  
 * tv  
 * theater  
 * movies  
 * concerts  
 * music  
 * shopping  
 * yoga  
 * interests_correlate: Correlation between participant’s and partner’s ratings of interests.  
 * expected_happy_with_sd_people: How happy do you expect to be with the people you meet during the speed-dating event?  
 * expected_num_interested_in_me: Out of the 20 people you will meet, how many do you expect will be interested in dating you?  
 * expected_num_matches: How many matches do you expect to get?  
 * like: Did you like your partner?  
 * guess_prob_liked: How likely do you think it is that your partner likes you?   
 * met: Have you met your partner before?  
 * decision: Decision at night of event.
 * decision_o: Decision of partner at night of event.  
 * match: Match (yes/no)'''
cols = cols.split('*')[1:] #Every column is pointed by an "*"
cols = [i.split(':')[0].strip() for i in cols] #Col names are located to the left of the colons
cols[:5] 

['gender', 'age', 'age_o', 'd_age', 'race']

In [7]:
cols_w_errors = [i for i in df.columns if i not in cols and not i.startswith('d_')] 
#The "d_" columns were added by the researchers and they don't have a description in the site.

In [8]:
cols_w_errors #has_null and wave don't belong to the experiment data either, as they were added to describe the dataset

['has_null', 'wave', 'pref_o_sincere', 'sinsere_o']

Only 'sinsere_o' needs to be fixed, since 'pref_o_sincere' is spelled wrong in the site. Also, "ambition_important" seems to have
a typo but it's incorrectly spelled in the site as well. I had to manually detect it.

In [17]:
df.rename(columns={'sinsere_o':'sincere_o','ambtition_important':'ambition_important'},inplace=True)

#### We end up with a cleaner dataset, but there is still some work to be done before it's ready for analysis.
We'll start by turning categorical columns into "one-hot encoding" format

In [18]:
df.info(verbose=True) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8378 entries, 0 to 8377
Data columns (total 123 columns):
 #    Column                           Dtype  
---   ------                           -----  
 0    has_null                         int8   
 1    wave                             int8   
 2    gender                           object 
 3    age                              float64
 4    age_o                            float64
 5    d_age                            int8   
 6    d_d_age                          object 
 7    race                             object 
 8    race_o                           object 
 9    samerace                         int8   
 10   importance_same_race             float64
 11   importance_same_religion         float64
 12   d_importance_same_race           object 
 13   d_importance_same_religion       object 
 14   field                            object 
 15   pref_o_attractive                float64
 16   pref_o_sincere                   float64

In [19]:
#We're only concerened with 'race','gender' and 'race_o', since 'field' has too many unique values and we'll drop it anyways.
df['field'].value_counts(normalize=True) 

Business                            0.062658
MBA                                 0.056284
Law                                 0.055562
'Social Work'                       0.045460
'International Affairs'             0.030307
                                      ...   
'Business [Finance & Marketing]'    0.000722
Stats                               0.000722
'MFA  Poetry'                       0.000722
'marine geophysics'                 0.000601
theory                              0.000601
Name: field, Length: 259, dtype: float64

In [20]:
new_cols= ['race_african_american','race_latino','race_european',
    'race_other','male','race_o_african_american',
    'race_o_latino','race_o_european','race_o_other']
df[new_cols] = pd.get_dummies(
    df[
        ['race','gender','race_o']
        ],
        drop_first=True,
    )
df[new_cols] #Looks fine

,race_african_american,race_latino,race_european,race_other,male,race_o_african_american,race_o_latino,race_o_european,race_o_other
0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
8373,0,0,1,0,1,0,1,0,0
8374,0,0,1,0,1,0,0,0,1
8375,0,0,1,0,1,0,1,0,0
8376,0,0,1,0,1,0,0,0,0


Now I can drop the original columns. 
I'll also be dropping columns containing "d_", since they were generated for the original research papers and will not be of use for our analysis.
Finally, descriptive columns such as "has_null" will also be removed

In [21]:
to_drop = ['race','gender','race_o','has_null','field']
to_drop.extend(col for col in df.columns if col.startswith('d_'))
df.drop(columns=to_drop,inplace=True)

Lastly, some columns were created in a way that is not helpful to our investigation. Those which contain information about each
participant's importance given to certain attributes. They were asked to rate 6 attributes (attractiveness, sincereness,
sense of humor, intelligence, ambition, and sharing interests) on a *shared* scale from 0 to 100. In order to have more objective ratings, we came up with a ranking system that narrows down the scales to only 1-6. In this case, 1 represents the highest rated attribute and 6 the lowest rated one. Ties are broken using the average. More details can be found in the "processing_tools" module.

In [22]:
#Let's check them out:
important_cols = [i for i in df.columns if 'important' in i]
df[important_cols]

,attractive_important,sincere_important,intellicence_important,funny_important,ambition_important,shared_interests_important
0,15.0,20.0,20.0,15.0,15.0,15.0
1,15.0,20.0,20.0,15.0,15.0,15.0
2,15.0,20.0,20.0,15.0,15.0,15.0
3,15.0,20.0,20.0,15.0,15.0,15.0
4,15.0,20.0,20.0,15.0,15.0,15.0
...,...,...,...,...,...,...
8373,70.0,0.0,15.0,15.0,0.0,0.0
8374,70.0,0.0,15.0,15.0,0.0,0.0
8375,70.0,0.0,15.0,15.0,0.0,0.0
8376,70.0,0.0,15.0,15.0,0.0,0.0


Now we transform them:

In [23]:
subset = df.dropna(subset=important_cols).copy() #Doesn't work with nans. We'll handle those in future notebooks.

In [24]:
subset[important_cols] = subset.apply(lambda x: rank_cols(
    x[important_cols[0]],x[important_cols[1]],
    x[important_cols[2]],x[important_cols[3]],
    x[important_cols[4]],x[important_cols[5]],
    keys=important_cols),axis=1).values.astype('float32')

In [25]:
#And now we just replace the original columns with the new ones
df[important_cols] = subset[important_cols]

And let's see the results:

In [26]:
df[important_cols]

,attractive_important,sincere_important,intellicence_important,funny_important,ambition_important,shared_interests_important
0,4.5,1.5,1.5,4.5,4.5,4.5
1,4.5,1.5,1.5,4.5,4.5,4.5
2,4.5,1.5,1.5,4.5,4.5,4.5
3,4.5,1.5,1.5,4.5,4.5,4.5
4,4.5,1.5,1.5,4.5,4.5,4.5
...,...,...,...,...,...,...
8373,1.0,5.0,2.5,2.5,5.0,5.0
8374,1.0,5.0,2.5,2.5,5.0,5.0
8375,1.0,5.0,2.5,2.5,5.0,5.0
8376,1.0,5.0,2.5,2.5,5.0,5.0


I repeat the same proceedure for the columns that correspond to the subject's partner:

In [27]:
important_o = [i for i in df.columns if 'pref_o' in i]
subset_2 = df.dropna(subset=important_o).copy() #Doesn't work with nans. We'll handle those in future notebooks.

In [28]:
subset_2[important_o] = subset_2.apply(lambda x: rank_cols(
    x[important_o[0]],x[important_o[1]],
    x[important_o[2]],x[important_o[3]],
    x[important_o[4]],x[important_o[5]],
    keys=important_o),axis=1).values.astype('float32')

In [29]:
df[important_o] = subset_2[important_o]

In [30]:
df[important_o]

,pref_o_attractive,pref_o_sincere,pref_o_intelligence,pref_o_funny,pref_o_ambitious,pref_o_shared_interests
0,1.0,3.0,3.0,3.0,6.0,5.0
1,1.0,4.5,4.5,2.0,4.5,4.5
2,1.5,3.5,1.5,3.5,5.0,6.0
3,2.0,5.0,3.0,1.0,5.0,5.0
4,1.0,5.0,2.5,5.0,5.0,2.5
...,...,...,...,...,...,...
8373,5.0,5.0,1.0,2.0,5.0,3.0
8374,1.0,2.0,3.5,5.5,3.5,5.5
8375,NaN,NaN,NaN,NaN,NaN,NaN
8376,5.0,1.5,1.5,5.0,5.0,3.0


#### Final steps before splitting into train and test sets

In [4]:
df = pd.read_parquet(save_path)

In [5]:
to_drop = ['like','match','wave']
to_drop.extend([i for i in df.columns if i.startswith('d_')])
df.drop(columns=to_drop,inplace=True) #Some columns are not useful for our experiment. Columns containing "d_", for example, don't add information to the models

We need to identify columns which can be rounded *after* imputing nulls

In [6]:
cols_to_round = spot_round_cols(df)

### Train-test split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['decision']).values,df['decision'].values, 
                                                    stratify=df[['decision','male']].values, 
                                                    test_size=0.1)
X_train = pd.DataFrame(X_train,columns=df.drop(columns=['decision']).columns)
y_train = pd.DataFrame(y_train, columns=['decision'])
X_test = pd.DataFrame(X_test,columns=df.drop(columns=['decision']).columns)
y_test = pd.DataFrame(y_test, columns=['decision'])
test = pd.concat([y_test,X_test],axis=1) #We set aside a validation set from the beginning, again, to avoid any kind of data leakage.
df = pd.concat([X_train,y_train],axis=1)

### Finally, null imputation. Very important to do it after splitting to avoid data leakage

Data imputation involves a certain degree of randomness. Because of this, results may differ a little from those pusblished in the paper

In [8]:
df = impute_nulls(df)
test = impute_nulls(test)

[IterativeImputer] Completing matrix with shape (7540, 67)
[IterativeImputer] Ending imputation round 1/10, elapsed time 183.18
[IterativeImputer] Change: 26.496752714007066, scaled tolerance: 0.055 
[IterativeImputer] Ending imputation round 2/10, elapsed time 359.43
[IterativeImputer] Change: 24.534499999999998, scaled tolerance: 0.055 
[IterativeImputer] Ending imputation round 3/10, elapsed time 547.52
[IterativeImputer] Change: 22.498400000000004, scaled tolerance: 0.055 
[IterativeImputer] Ending imputation round 4/10, elapsed time 748.07
[IterativeImputer] Change: 21.1083, scaled tolerance: 0.055 
[IterativeImputer] Ending imputation round 5/10, elapsed time 943.54
[IterativeImputer] Change: 20.4061, scaled tolerance: 0.055 
[IterativeImputer] Ending imputation round 6/10, elapsed time 1157.57
[IterativeImputer] Change: 23.4381, scaled tolerance: 0.055 
[IterativeImputer] Ending imputation round 7/10, elapsed time 1389.96
[IterativeImputer] Change: 22.651499999999995, scaled tol

c:\Users\Laureano\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\impute\_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
c:\Users\Laureano\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:402: UserWarning: X has feature names, but IterativeImputer was fitted without feature names
  warnings.warn(


[IterativeImputer] Ending imputation round 1/10, elapsed time 2.81
[IterativeImputer] Ending imputation round 2/10, elapsed time 5.56
[IterativeImputer] Ending imputation round 3/10, elapsed time 8.16
[IterativeImputer] Ending imputation round 4/10, elapsed time 11.01
[IterativeImputer] Ending imputation round 5/10, elapsed time 13.66
[IterativeImputer] Ending imputation round 6/10, elapsed time 16.57
[IterativeImputer] Ending imputation round 7/10, elapsed time 19.04
[IterativeImputer] Ending imputation round 8/10, elapsed time 21.34
[IterativeImputer] Ending imputation round 9/10, elapsed time 23.78
[IterativeImputer] Ending imputation round 10/10, elapsed time 26.05
[IterativeImputer] Completing matrix with shape (838, 67)
[IterativeImputer] Ending imputation round 1/10, elapsed time 24.95
[IterativeImputer] Change: 16.948134260685354, scaled tolerance: 0.055 
[IterativeImputer] Ending imputation round 2/10, elapsed time 50.81
[IterativeImputer] Change: 12.439700000000002, scaled to

c:\Users\Laureano\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\impute\_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
c:\Users\Laureano\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:402: UserWarning: X has feature names, but IterativeImputer was fitted without feature names
  warnings.warn(


[IterativeImputer] Ending imputation round 1/10, elapsed time 0.77
[IterativeImputer] Ending imputation round 2/10, elapsed time 1.56
[IterativeImputer] Ending imputation round 3/10, elapsed time 2.35
[IterativeImputer] Ending imputation round 4/10, elapsed time 3.14
[IterativeImputer] Ending imputation round 5/10, elapsed time 3.99
[IterativeImputer] Ending imputation round 6/10, elapsed time 4.79
[IterativeImputer] Ending imputation round 7/10, elapsed time 5.56
[IterativeImputer] Ending imputation round 8/10, elapsed time 6.35
[IterativeImputer] Ending imputation round 9/10, elapsed time 7.12
[IterativeImputer] Ending imputation round 10/10, elapsed time 7.88


### Rounding

In [13]:
for col in cols_to_round: 
    try:
        test.loc[:,(col)] = np.round(test[col]).astype('int8') 
        df.loc[:,(col)] = np.round(df[col]).astype('int8')
    except:
        print(col)


In [14]:
df['met'] = np.where(df['met']>1,1,df['met']) #Setting a limit manually for some columns (Hard to set specific columns on the sklearn imputer)
test['met'] = np.where(test['met']>1,1,test['met'])
for i in ['funny_o','gaming','reading']:
    df[i] = np.where(df[i]>10,10,df[i])
    test[i] = np.where(test[i]>10,10,test[i])

A duplicate of the column "decision" was created in the train-test split. We'll simply remove it 

In [24]:

test.loc[:,test.columns.duplicated()]

,decision
0,0
1,0
2,0
3,0
4,0
...,...
833,0
834,1
835,1
836,1


In [25]:
test = test.loc[:,~test.columns.duplicated()]


The datasets are now clean enough for further analysis. I'll save the results in parquet format to keep the object types 
(plus it's a bit more memory-efficient).

In [26]:
train_path = '../Datasets//df_imput.parquet'
test_path = '../Datasets//test_imput.parquet'
df.to_parquet(train_path)
test.to_parquet(test_path)
